In [49]:
import numpy as np
import pandas as pd
from datetime import datetime

In [50]:
# read raw data
raw_df = pd.read_csv('inputs/raw_data.csv')
print(len(raw_df))
raw_df.head()

3897


,name,position,height,matches_year1,goals_year1,matches_year2,goals_year2,matches_year3,goals_year3,prediction_season
0,Tim Kneule,Middle Back,190,33,31,10,0,31,76,2012/2013
1,Tim Kneule,Middle Back,190,10,0,31,76,33,107,2013/2014
2,Tim Kneule,Middle Back,190,31,76,33,107,34,83,2014/2015
3,Tim Kneule,Middle Back,190,33,107,34,83,34,71,2015/2016
4,Tim Kneule,Middle Back,190,34,83,34,71,26,74,2016/2017


In [51]:
# filter raw data
raw_df = raw_df.dropna()
raw_df = raw_df[(raw_df['height'] >= 120) & (raw_df['height'] <= 230)]
raw_df = raw_df[(raw_df['matches_year1'] > 0) & (raw_df['matches_year2'] > 0) & (raw_df['matches_year3'] > 0)]
raw_df = raw_df[(raw_df['goals_year1'] >= 0) & (raw_df['goals_year2'] >= 0) & (raw_df['goals_year3'] >= 0)]
raw_df = raw_df[raw_df['position'].isin(['Middle Back','Left Wing','Right Back','Right Wing','Line Player',\
                                         'Left Back','Back'])]
raw_df = raw_df[raw_df['prediction_season'] != '2018/2019']
print(raw_df['position'].unique())
print(len(raw_df))
raw_df.head()

['Middle Back' 'Left Wing' 'Right Back' 'Right Wing' 'Line Player'
 'Left Back' 'Back']
3689


,name,position,height,matches_year1,goals_year1,matches_year2,goals_year2,matches_year3,goals_year3,prediction_season
0,Tim Kneule,Middle Back,190,33,31,10,0,31,76,2012/2013
1,Tim Kneule,Middle Back,190,10,0,31,76,33,107,2013/2014
2,Tim Kneule,Middle Back,190,31,76,33,107,34,83,2014/2015
3,Tim Kneule,Middle Back,190,33,107,34,83,34,71,2015/2016
4,Tim Kneule,Middle Back,190,34,83,34,71,26,74,2016/2017


In [52]:
def is_center(position):
    if position == 'Middle Back':
        return 1
    else:
        return 0

def is_back(position):
    if position == 'Right Back' or position == 'Left Back' or position == 'Back':
        return 1
    else:
        return 0
    
def is_wing(position):
    if position == 'Right Wing' or position == 'Left Wing':
        return 1
    else:
        return 0
    
def is_line(position):
    if position == 'Line Player':
        return 1
    else:
        return 0

In [53]:
# build necessary columns
raw_df['goals_per_match_year1'] = raw_df['goals_year1'] / raw_df['matches_year1']
raw_df['goals_per_match_year2'] = raw_df['goals_year2'] / raw_df['matches_year2']
raw_df['goals_per_match_year3'] = raw_df['goals_year3'] / raw_df['matches_year3']

# create binary features for field position
raw_df['center'] = raw_df.apply(lambda row: is_center(row['position']), axis=1)
raw_df['back'] = raw_df.apply(lambda row: is_back(row['position']), axis=1)
raw_df['wing'] = raw_df.apply(lambda row: is_wing(row['position']), axis=1)
raw_df['line'] = raw_df.apply(lambda row: is_line(row['position']), axis=1)

raw_df = raw_df[['name','center','back','wing','line','height','matches_year1','goals_year1',\
                 'goals_per_match_year1','matches_year2','goals_year2','goals_per_match_year2',\
                 'goals_per_match_year3']]
raw_df.to_csv('inputs/filtered_data_before_median_filtering.csv', sep=',', index=False)
print(len(raw_df))
raw_df.head()

3689


,name,center,back,wing,line,height,matches_year1,goals_year1,goals_per_match_year1,matches_year2,goals_year2,goals_per_match_year2,goals_per_match_year3
0,Tim Kneule,1,0,0,0,190,33,31,0.939394,10,0,0.000000,2.451613
1,Tim Kneule,1,0,0,0,190,10,0,0.000000,31,76,2.451613,3.242424
2,Tim Kneule,1,0,0,0,190,31,76,2.451613,33,107,3.242424,2.441176
3,Tim Kneule,1,0,0,0,190,33,107,3.242424,34,83,2.441176,2.088235
4,Tim Kneule,1,0,0,0,190,34,83,2.441176,34,71,2.088235,2.846154


In [54]:
# median filtering
medians = raw_df.groupby(['center','back','wing','line','height','matches_year1','goals_year1','matches_year2',
                          'goals_year2','goals_per_match_year1','goals_per_match_year2']).goals_per_match_year3\
                          .median()

medians = medians.reset_index()
medians['N'] = 0

for index, row in medians.iterrows():
    medians.loc[index, 'N'] = raw_df.loc[(raw_df['center'] == row['center']) & (raw_df['back'] == row['back']) &\
    (raw_df['wing'] == row['wing']) & (raw_df['line'] == row['line']) & (raw_df['height'] == row['height']) & \
    (raw_df['matches_year1'] == row['matches_year1']) & (raw_df['goals_year1'] == row['goals_year1']) & \
    (raw_df['goals_per_match_year1'] == row['goals_per_match_year1']) & \
    (raw_df['matches_year2'] == row['matches_year2']) & (raw_df['goals_year2'] == row['goals_year2']) & \
    (raw_df['goals_per_match_year2'] == row['goals_per_match_year2'])].shape[0]

df = medians

print ("Medians filtering:",df.shape[0],"values","\n")
print (df['height'].describe(percentiles=[.25,.50,.75]),"\n")    
print (df['matches_year1'].describe(percentiles=[.25,.50,.75]),"\n")
print (df['goals_year1'].describe(percentiles=[.25,.50,.75]),"\n")
print (df['goals_per_match_year1'].describe(percentiles=[.25,.50,.75]),"\n")
print (df['matches_year2'].describe(percentiles=[.25,.50,.75]),"\n")
print (df['goals_year2'].describe(percentiles=[.25,.50,.75]),"\n")
print (df['goals_per_match_year2'].describe(percentiles=[.25,.50,.75]),"\n")
print (df['N'].describe(percentiles=[.25,.50,.75]),"\n")
print (df.head())

Medians filtering: 3685 values 

count    3685.000000
mean      186.312347
std         9.572875
min       159.000000
25%       180.000000
50%       188.000000
75%       194.000000
max       212.000000
Name: height, dtype: float64 

count    3685.000000
mean       24.236092
std         9.135614
min         1.000000
25%        20.000000
50%        26.000000
75%        31.000000
max        48.000000
Name: matches_year1, dtype: float64 

count    3685.000000
mean       64.980461
std        50.279119
min         0.000000
25%        24.000000
50%        57.000000
75%        95.000000
max       279.000000
Name: goals_year1, dtype: float64 

count    3685.000000
mean        2.482818
std         1.763655
min         0.000000
25%         1.190476
50%         2.300000
75%         3.500000
max        41.000000
Name: goals_per_match_year1, dtype: float64 

count    3685.000000
mean       25.417639
std         8.254704
min         1.000000
25%        22.000000
50%        26.000000
75%        31.0000

In [55]:
# save final processed data to file
df.to_csv('inputs/final_processed_data.csv', sep=',', index=False)